# クイックスタート2

このノートのクイックスタートは、2025年2月時点での Microsoft の Azure AI Foundry の[ドキュメント](https://learn.microsoft.com/ja-jp/azure/ai-studio/quickstarts/get-started-code)に記載されている内容になります。

※仕様やドキュメントの変更に伴い、この内容が古くなる可能性があります。


## 前提条件

このノートの内容を学習するにあたって、以下が必要になります。

- Visial Studio Code または このノートブックを実行可能な Jupyter が動作する環境
- Azure サブスクリプション
- 作成済みの Azure AI Foundry プロジェクトとデプロイしたモデル
- Python 3.8 以降
- Python に関する基礎知識
- 必要な Python パッケージ（下記のコードでインストールする）


## 概要：Azure AI Foundry SDK を使用して Python で簡単なチャット アプリを構築する (2)

このノートでは、Azure AI Foundry SDK を使用して、簡単なチャットアプリケーションを開発する方法について説明します。

このクイックスタートでは、クイックスタート1 の続きとして、`PromptTemplate` を利用します。


### 準備

このクイック スタートを始める前に、Azure AI Foundry ポータルから、ハブ リソースとプロジェクト リソースを作成して `gpt-4o-mini` モデルをハブまたはプロジェクトへデプロイしてください。

> [Azure AI Foundry プレイグラウンドのクイックスタート](https://learn.microsoft.com/ja-jp/azure/ai-studio/quickstarts/get-started-playground) を完了している場合、必要な作業は完了しています。

Visual Studio Code でこのノートを開き、[ドキュメントのこのセクション](https://code.visualstudio.com/docs/python/environments#_creating-environments) を参考にしながら、ワークスペースに Python 仮想環境を作成します。仮想環境を作成する際には、このフォルダにある `requirements.txt` を選択すると、このノートの Python アプリケーションコードの実行に必要なライブラリを Python 仮想環境にインストールすることができます。

仮想環境の準備ができたら、このノートにある Python コードをステップごとに実行します。

### チャットアプリのコードサンプル

#### 1. クイックスタート 1 で使ったコードを使います

  - `.env` ファイルの内容を環境変数に読み込みます。
  - `AIProjectClient` の `from_connection_string()` を使って、Azure AI Foundry プロジェクトに接続します。
  - `get_chat_completions_client()` を使って、チャット補完クライアント `chat` (`ChatCompletionsClient` クラスのインスタンス) を作成します。


In [ ]:
from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential

import os
from dotenv import load_dotenv

load_dotenv('.env', override=True)
project_connection_string = os.getenv('PROJECT_CONNECTION_STRING')
model_name_string = os.getenv('MODEL_NAME')

project = AIProjectClient.from_connection_string(
    conn_str=project_connection_string, credential=DefaultAzureCredential()
)

chat = project.inference.get_chat_completions_client()

#### 2. プロンプト テンプレートを作成する

推論モデルに問い合わせるときに利用するシステムプロンプトを、プロンプトテンプレート文字列として定義します。プロンプトテンプレートには可変項目を含めることができ、動的なプロンプトを構築できます。このプログラムでは、プロンプトテンプレートを作成して、チャットクライアントに渡します。

In [ ]:
prompt_template_string="""
  system:
  あなたは 2350 年のテクノ パンク ロッカーのように話す AI アシスタントです。クールに、でもクール過ぎないようにしましょう。わかりましたか??
  ユーザーをファーストネームで呼び、ラストネームをダジャレにしてみてください。

  ユーザーのファーストネームは {{first_name}}、ラストネームは {{last_name}} です
"""

In [ ]:
prompt_template_string="""
  system:
  あなたはグルメと旅行に詳しいコンシェルジュです。質問に対して現在の流行とその詳細な情報を付け加えて説明してください。わかりましたか??"
  ユーザーをラストネームに「様」を付けて呼んでください。

  ユーザーのファーストネームは {{first_name}}、ラストネームは {{last_name}} です"
"""

### 3. 推論モデルからの回答を取得する関数を定義します

`PromptTemple` の `from_string()` を使ってプロンプトテンプレートを作成します。
プロンプトテンプレートの可変項目に対するコンテキストを受け取り、`create_message()` を使って、チャット補完クライアントに渡すシステムプロンプトを作成します。

システムプロンプトと問い合わせメッセージを組み合わせて、チャット補完クライアントを使って問い合わせを行い回答を取得 (`complete()`) します。

In [ ]:
from azure.ai.inference.prompts import PromptTemplate

def get_chat_response(messages, context):
    # create a prompt template from an inline string (using mustache syntax)
    prompt_template = PromptTemplate.from_string(prompt_template_string)

    # generate system message from the template, passing in the context as variables
    system_message = prompt_template.create_messages(data=context)

    # add the prompt messages to the user messages
    return chat.complete(
        model=model_name_string,
        messages=system_message + messages,
        temperature=1,
        frequency_penalty=0.5,
        presence_penalty=0.5,
    )

#### 4. 推論モデルへの質問を作成します

In [ ]:
query = "世界で一番おいしい食べ物がある都市はどこですか?"
context={"first_name": "一郎", "last_name": "佐藤"}

In [ ]:
query = "世界で一番綺麗な場所がある都市はどこですか?"
context={"first_name": "五郎", "last_name": "田中"}

#### 5. アプリケーションを実行します

`__main__` 関数を呼び出し、アプリケーションを実行します。

In [ ]:
from azure.ai.inference.models import UserMessage

if __name__ == "__main__":
    response = get_chat_response(
        messages=[UserMessage(content=query)],
        context=context,
    )
    print(response.choices[0].message.content)
    print(response)